# Hadling large dataset by PySpark

## Requirements

In [2]:
!pip install pyspark
!pip install findspark
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Checking information of session

In [3]:
import pandas as pd
import pyspark

import findspark
findspark.init()

In [4]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
print(spark)

In [5]:
from pyspark import SparkContext
from pyspark.sql import SQLContext 

s_context = SparkContext.getOrCreate()
#s_context = SparkContext("local", "existencia_colmenas")
sql = SQLContext(s_context)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Reading Excel files

In [6]:
import os

class FileReader:
  def __init__(self, path:str):
    self.path = path
    self.list_f = []
    self.collect_files()
  
  def collect_files(self):
    # Get the list of all files and directories
    self.list_f = os.listdir(self.path)
    self.list_f = [ self.path +'/'+ x for x in self.list_f ]

  def show_files(self):
    for x in range(len(self.list_f)):
      print(x+1, "  ", self.list_f[x])

In [7]:
folder = FileReader("./data_income")
folder.show_files()

1    ./data_income/numero-superficie-de-upa-forestal-regional-comunal.xlsx
2    ./data_income/numero-personas-administradoras.xlsx
3    ./data_income/numero-superficie-de-upa-bosque-nativo-regional-comunal.xlsx
4    ./data_income/superficie-categoría-cultivo-región-comuna.xlsx
5    ./data_income/numero-superficie-de-upa-frutales-regional-comunal.xlsx
6    ./data_income/tamaño-upa-región-comuna.xlsx
7    ./data_income/existencias-animales.xlsx
8    ./data_income/actividad-principal-región.xlsx
9    ./data_income/01_numero_superficie_de_upa_censadas_regional-xlsx.xlsx
10    ./data_income/numero-superficie-de-upa-aire-libre-bajo-cubierta-regional-comunal.xlsx
11    ./data_income/numero-upa-orientación-colmenas.xlsx
12    ./data_income/tipo-gestión-región-comuna.xlsx
13    ./data_income/numero-upa-practicas-mejoramiento-suelo.xlsx
14    ./data_income/superficie-principal-sistema-riego.xlsx
15    ./data_income/numero-productores-pueblos-originarios.xlsx
16    ./data_income/numero-de-upa-y-s

## 3. Reading data

In [62]:
from pyspark.sql import functions as F
from functools import reduce
import numpy as np
import unidecode

class spark_dataframe:
  def __init__(self, path_origin:str, path_export:str):
    """Standardising all data types from pandas to String for spark"""
    df = pd.read_excel(path_origin, sheet_name=0)
    df = df.astype(str)
    self.df_spark = sql.createDataFrame(df)
    self.df_spark.show(n=20)
    self.path_export = path_export
    self.path_origin = path_origin

  def remove_nan(self):
    """Replace nan with Null"""
    self.df_spark = self.df_spark.replace('nan', None)
    self.df_spark.show(n=20)
  
  def remove_comments(self):
    """Drop rows with null values on selected columns
       ---Removing comments---"""
    columns = self.df_spark.columns
    self.df_spark = self.df_spark.na.drop(
                  subset=columns[2:],how="all"
                )
    self.df_spark.show(n=20)

  def remove_empty_column(self):
    """ Drop columns with null values"""
    null_counts = self.df_spark.select(
                    [
                        F.count(
                            F.when(
                                F.col(a).isNull(), a
                                )
                            ).alias(a) 
                        for a in self.df_spark.columns
                    ]
                  ).collect()[0].asDict()

    to_delete = [ p for p, u in null_counts.items() if u == self.df_spark.count() ]

    self.df_spark = self.df_spark.drop(*to_delete)
    self.df_spark.show(truncate=True, n=20)
  
  def get_header_dataframe(self):
    """Getting those columns with null field (in other words, headers)"""
    print(">>>>>>>>>>  >>>>>>>>>>  >>>>>>>>>>  Proceso desconocido")
    cols = [F.col(c) for c in self.df_spark.columns]
    
    filter_expr = reduce(
        lambda a, b: a | b.isNull(), 
        cols[1:], 
        cols[0].isNull()
        )
    #filter_expr = F.exists(F.array(*self.df_spark.columns), lambda x: x.isNull())
    head_df = self.df_spark.filter(filter_expr)
    row_reduction = head_df.count()
    if row_reduction >1:
      self.df_spark = spark.createDataFrame(self.df_spark.tail(self.df_spark.count() - row_reduction), self.df_spark.schema)

    
    
    #########################33
    #query = []
    #for col in self.df_spark.columns:
    #  query.append("'"+col+"' IS NULL ")
    #query = ' OR '.join(query)
#
    #print(query)
    #
    #head_df = self.df_spark.filter(query)
    #head_df.show()
    #print(type(head_df))
    #########################33

    if head_df.count()==1:
      rows = self.df_spark.take(2)
      head_df = spark.createDataFrame(rows)
      """ Removing two first row"""
      self.df_spark = spark.createDataFrame(self.df_spark.tail(self.df_spark.count()-2), self.df_spark.schema)

    if head_df.isEmpty():
      expr = [F.first(col).alias(col) for col in self.df_spark.columns]
      head_df = self.df_spark.agg(*expr)
      """ Removing first row"""
      self.df_spark = spark.createDataFrame(self.df_spark.tail(self.df_spark.count()-1), self.df_spark.schema)


    
    head_df.show(truncate=False,n=20)

    return head_df
  ################################################################################################################
  def filling_data(self, multi_array): 
    for row in multi_array:
      last_value=None
      for i in range(len(row)):
        if ( row[i]==None and i>0 ):
          row[i] = last_value
        else:
          last_value = row[i]
    return multi_array

  def data_normalization(self, xl_array):  
    """It will swap: uppercase to lowercase, 
    blanckspace to underscore, and other"""
    xl_array = np.char.lower(xl_array)
    xl_array = np.char.strip(xl_array)
    xl_array = np.char.replace(xl_array, "-", "")
    xl_array = np.char.replace(xl_array, "  ", " ")
    xl_array = np.char.replace(xl_array, " ", "_")
    xl_array = np.char.replace(xl_array, "none", "")
    xl_array = np.char.replace(xl_array, ",", "")
    xl_array = np.char.replace(xl_array, "\n", "_")
    xl_array = np.char.replace(xl_array, "(", "")
    xl_array = np.char.replace(xl_array, ")", "")
    ##self.xl_array = np.char.replace(self.xl_array, ".", "")
    ##self.xl_array = np.char.replace(self.xl_array, ",", "")
    return xl_array

  # Removing non ascii and special characteres in columns
  def ascii_ignore(self, x):
    return unidecode.unidecode(x)

  # Mergin columns to unique ones
  def mergin_rows(self, array):
    a = []
    for row in array:
      a.append(np.char.array(row))
    b =np.char.array(0)
    for x in a:
      b = b + "__" +x
    b = b.replace('0___', '')
    b = b.replace('0_', '')
    b = [ self.ascii_ignore(x) for x in b ]
    return b
  ################################################################################################################
  
  def get_header_np(self):
    header_df = self.get_header_dataframe()
    
    print("entrando")

    header_np = np.array(header_df.select( header_df.columns ).collect())
    header_np = self.mergin_rows(self.data_normalization(self.filling_data(header_np).astype(str)))
    self.header_np = header_np

  def remove_any_null_rows(self):
    """removing whatever rows that contains null field"""
    self.df_spark = self.df_spark.na.drop(how="any")
    self.df_spark.show(truncate=False,n=20)

  def Establishing_header(self):
    
    columns = self.df_spark.columns
    self.get_header_np()

    print(self.header_np)

    for old, new in zip(columns, self.header_np):
      self.df_spark = self.df_spark.withColumnRenamed(old, new)

    #self.df_spark = self.df_spark.toDF(*self.header_np)
    self.df_spark.show(truncate=False,n=10)

  def export_to_csv(self):
    self.df_spark.write.options(header='True', delimiter=',').format("csv").mode('overwrite').save(self.path_export + self.path_origin.split('/')[-1].split('.')[0] )


## Funtionality Testing

In [ ]:
first = spark_dataframe(path_origin=folder.list_f[0], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
first.Establishing_header()
first.remove_any_null_rows()
first.export_to_csv()

In [ ]:
first = spark_dataframe(path_origin=folder.list_f[1], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
first.Establishing_header()

#first.remove_any_null_rows()
first.export_to_csv()

In [ ]:
first = spark_dataframe(path_origin=folder.list_f[2], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
#first.remove_any_null_rows()
first.Establishing_header()
first.export_to_csv()

In [ ]:
first = spark_dataframe(path_origin=folder.list_f[3], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
#first.remove_any_null_rows()
first.Establishing_header()
first.export_to_csv()

In [ ]:
first = spark_dataframe(path_origin=folder.list_f[4], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
#first.remove_any_null_rows()
first.Establishing_header()
first.export_to_csv()

In [64]:
first = spark_dataframe(path_origin=folder.list_f[5], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
#first.remove_any_null_rows()
first.Establishing_header()
first.export_to_csv()

+----------+--------------------+--------------------+--------------+-----------------+----------------+---------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+---------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+-----------------+-----------------+
|Unnamed: 0|          Unnamed: 1|          Unnamed: 2|    Unnamed: 3|       Unnamed: 4|      Unnamed: 5|     Unnamed: 6|          Unnamed: 7|       Unnamed: 8|          Unnamed: 9|      Unnamed: 10|         Unnamed: 11|      Unnamed: 12|         Unnamed: 13|      Unnamed: 14|         Unnamed: 15|      Unnamed: 16|         Unnamed: 17|    Unnamed: 18|         Unnamed: 19|      Unnamed: 20|         Unnamed: 21|      Unnamed: 22|         Unnamed: 23|      Unnamed: 24|      Unnamed: 25|      U

In [63]:
first = spark_dataframe(path_origin=folder.list_f[6], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
#first.remove_any_null_rows()
first.Establishing_header()
first.export_to_csv()

+----------+--------------------+--------------------+----------+----------+----------+----------+--------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+------------------+-----------+--------------------+
|Unnamed: 0|          Unnamed: 1|          Unnamed: 2|Unnamed: 3|Unnamed: 4|Unnamed: 5|Unnamed: 6|          Unnamed: 7|Unnamed: 8|Unnamed: 9|Unnamed: 10|Unnamed: 11|Unnamed: 12|Unnamed: 13|Unnamed: 14|       Unnamed: 15|Unnamed: 16|         Unnamed: 17|
+----------+--------------------+--------------------+----------+----------+----------+----------+--------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+------------------+-----------+--------------------+
|       nan|12.  Existencias ...|                 nan|       nan|       nan|       nan|       nan|                 nan|       nan|       nan|        nan|        nan|        nan|        nan|        nan|               nan|        nan|      

In [65]:
first = spark_dataframe(path_origin=folder.list_f[7], path_export="./data_outcome/")
first.remove_nan()
first.remove_comments()
first.remove_empty_column()
#first.remove_any_null_rows()
first.Establishing_header()
first.export_to_csv()

+----------+--------------------+----------------+--------------------+----------+----------+--------------------+----------+--------------------+------------------+-------------+--------------------+--------------------+
|Unnamed: 0|          Unnamed: 1|      Unnamed: 2|          Unnamed: 3|Unnamed: 4|Unnamed: 5|          Unnamed: 6|Unnamed: 7|          Unnamed: 8|        Unnamed: 9|  Unnamed: 10|         Unnamed: 11|         Unnamed: 12|
+----------+--------------------+----------------+--------------------+----------+----------+--------------------+----------+--------------------+------------------+-------------+--------------------+--------------------+
|       nan|2. Número de Unid...|             nan|                 nan|       nan|       nan|                 nan|       nan|                 nan|               nan|          nan|                 nan|                 nan|
|       nan|                 nan|             nan|                 nan|       nan|       nan|                 na